In [ ]:
import os
from PIL import Image, ImageEnhance, ImageOps
import random
from random import randint
import matplotlib.pyplot as plt
import numpy as np
from tqdm import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
import tensorflow as tf;
from training_set_utils import *
from image_utils import *

print(tf.version)

atlas_dir = "images/atlas/"
input_width = 28
input_height = 28
output_classes = 10

In [ ]:
%matplotlib inline
print("loading training data from " + atlas_dir)
training_set = create_from_atlas_dir(atlas_dir)

In [ ]:
%matplotlib inline
print("creating variations")
training_set_ext = create_variations(training_set)

In [ ]:
%matplotlib inline
print("converting training data")

random.shuffle(training_set_ext)

def img_to_nparray(image):
    image = image.convert("L")
    image = image.resize((input_width, input_height), Image.LANCZOS)
    return np.asarray(image).astype(np.float32) / 255.0

test_samples = 1000
x_train_arr = []
y_train_arr = []
x_test_arr = []
y_test_arr = []
for i, item in enumerate(tqdm(training_set_ext)):
    if i < test_samples:
        x_test_arr.append(img_to_nparray(item.image))
        y_test_arr.append(np.asarray(item.y).astype(np.float32))
    else:
        x_train_arr.append(img_to_nparray(item.image))
        y_train_arr.append(np.asarray(item.y).astype(np.float32))
    
# convert to numpy types
x_train = np.asarray(x_train_arr)
y_train = np.asarray(y_train_arr)
x_test = []
y_test = []
if test_samples > 0:
    x_test = np.asarray(x_test_arr)
    y_test = np.asarray(y_test_arr)

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation = 'relu', input_shape = (input_height, input_width, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_classes, activation = 'softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [ ]:
model = createModel()
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import time

use_generator = False
batch_size = 32 * 1024

gen = ImageDataGenerator(
            width_shift_range=0, 
            height_shift_range=0, 
            zoom_range=0.1, 
            horizontal_flip=False)
            #rotation_range=10,
            #brightness_range=[0.5, 1.0])

for i in range(99999):
    
    #use_generator = (i % 2 == 1)
    
    if use_generator:        
        model.fit_generator(gen.flow(
            x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1), 
            y_train, batch_size=batch_size, shuffle=True), epochs=1, workers=24, steps_per_epoch=len(x_train) / batch_size)
    else:
        model.fit(
            x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1),
            y_train,
            epochs=50,
            batch_size=batch_size)
        
    loss_acc = model.evaluate(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1),
        y_test,
        batch_size=batch_size)
    
    model.save_weights(str(int(time.time())) + "_weights_" + str(input_width) + "x" + str(input_height) + "_c" + str(output_classes) + "_acc" + str(loss_acc[1]) + ".h5")

In [ ]:
def argmax2(arr):
    idx0 = 0
    last_max = 0.0
    for i in range(len(arr)):
        if arr[i] > last_max:
            last_max = arr[i]
            idx0 = i    
    idx1 = 0
    last_max = 0.0
    for i in range(len(arr)):
        if i != idx0 and arr[i] > last_max:
            last_max = arr[i]
            idx1 = i
    return (idx0, idx1)

def confidence(arr, pred):
    conv0 = arr[pred[0]]
    conv1 = arr[pred[1]]
    return (round(conv0, 2), round(conv1, 2))

def predict(img):
    result = model.predict(img.reshape(1, img.shape[0], img.shape[1], 1))[0]
    pred = argmax2(result)
    return (pred, confidence(result, pred))

def predict_test(idx):
    result = predict(x_test[idx])
    pred = result[0]
    conf = result[1]
    expect = argmax2(y_test_arr[idx])
    expect_conf = confidence(y_test_arr[idx], expect)
    if expect_conf[0] > 0.6:
        correct = expect[0] == pred[0]
        print("correct: " + str(correct) + 
              "; expected: " + str(expect) + " " + str(expect_conf) +
              "; predicted: " + str(pred) + " " + str(conf))
        if not correct: # or convidence < 0.9:
            training_set_ext[idx].show()
            #plt.imshow(x_test[idx])
            #plt.show()
    else:
        print("skip")
for i in range(len(x_test_arr)):
    predict_test(i)

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
filename = "ocr_model_" + str(input_width) + "x" + str(input_height) + "_c" + str(output_classes)
print("writing " + filename + ".tflite")
open(filename + ".tflite", "wb").write(tflite_model)
print("Now call the following in git bash: xxd -i " + filename + ".tflite > ../src/" + filename + ".c")